In [1]:
import requests
from bs4 import BeautifulSoup as soup
import re
import pandas as pd

In [2]:
current_url = 'http://books.toscrape.com/catalogue/page-1.html'
base_url = 'http://books.toscrape.com/catalogue/'

res = requests.get(current_url)
data = soup(res.text , 'html.parser')
next = data.find(class_ = 'next')
base_url+next.a['href']

'http://books.toscrape.com/catalogue/page-2.html'

In [3]:
base_url = 'http://books.toscrape.com/catalogue/'

current_url = 'http://books.toscrape.com/catalogue/page-1.html'
res = requests.get(current_url)

all_urls = ['http://books.toscrape.com/catalogue/page-1.html']

while(res.status_code == 200):
    data = soup(res.text , 'html.parser')
    
    next_page = data.find(class_ = 'next')
    if next_page is None:
        break
    next_page_url = base_url + next_page.a['href']
    all_urls.append(next_page_url)
    
    current_url = next_page_url
    res = requests.get(current_url)

print(len(all_urls))

50


In [4]:
all_urls[:5]

['http://books.toscrape.com/catalogue/page-1.html',
 'http://books.toscrape.com/catalogue/page-2.html',
 'http://books.toscrape.com/catalogue/page-3.html',
 'http://books.toscrape.com/catalogue/page-4.html',
 'http://books.toscrape.com/catalogue/page-5.html']

In [5]:
response = requests.get(all_urls[1])
data = soup(response.text , 'html.parser')
title = data.find_all(class_ = 'product_pod')
for i in title[:3]:
    print(i.h3.a['title'])

In Her Wake
How Music Works
Foolproof Preserving: A Guide to Small Batch Jams, Jellies, Pickles, Condiments, and More: A Foolproof Guide to Making Small Batch Jams, Jellies, Pickles, Condiments, and More


In [7]:
price = data.find_all(class_ = 'price_color')
for i in price[:3]:
    print(i.string[1:])

£12.84
£37.32
£30.52


In [8]:
image = data.find_all(class_ = 'image_container')
for i in image[:3]:
    print(i.img['src'])

../media/cache/5d/72/5d72709c6a7a9584a4d1cf07648bfce1.jpg
../media/cache/5c/c8/5cc8e107246cb478960d4f0aba1e1c8e.jpg
../media/cache/9f/59/9f59f01fa916a7bb8f0b28a4012179a4.jpg


In [9]:
all_image_urls = []
all_titles = []
all_prices = []

for a in all_urls:
    
    response = requests.get(a)
    data = soup(response.text , 'html.parser')
    
    img_link = data.find_all(class_ = 'image_container')
    for i in img_link:
        all_image_urls.append(i.img['src'])
    
    title = data.find_all(class_ = 'product_pod')
    for j in title:
        all_titles.append(j.h3.a['title'])
    
    price = data.find_all(class_ = 'price_color')
    for k in price:
        all_prices.append(k.string[1:])   

In [10]:
len(all_image_urls)

1000

In [11]:
all_image_urls[:5]

['../media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg',
 '../media/cache/26/0c/260c6ae16bce31c8f8c95daddd9f4a1c.jpg',
 '../media/cache/3e/ef/3eef99c9d9adef34639f510662022830.jpg',
 '../media/cache/32/51/3251cf3a3412f53f339e42cac2134093.jpg',
 '../media/cache/be/a5/bea5697f2534a2f86a3ef27b5a8c12a6.jpg']

In [12]:
len(all_titles)

1000

In [13]:
all_titles[:5]

['A Light in the Attic',
 'Tipping the Velvet',
 'Soumission',
 'Sharp Objects',
 'Sapiens: A Brief History of Humankind']

In [14]:
len(all_prices)

1000

In [15]:
all_prices[:5]

['£51.77', '£53.74', '£50.10', '£47.82', '£54.23']

In [16]:
df = pd.DataFrame({'image_url':all_image_urls , 'book_title':all_titles , 'product_price':all_prices})

In [17]:
df.head()

,image_url,book_title,product_price
0,../media/cache/2c/da/2cdad67c44b002e7ead0cc356...,A Light in the Attic,£51.77
1,../media/cache/26/0c/260c6ae16bce31c8f8c95dadd...,Tipping the Velvet,£53.74
2,../media/cache/3e/ef/3eef99c9d9adef34639f51066...,Soumission,£50.10
3,../media/cache/32/51/3251cf3a3412f53f339e42cac...,Sharp Objects,£47.82
4,../media/cache/be/a5/bea5697f2534a2f86a3ef27b5...,Sapiens: A Brief History of Humankind,£54.23


In [18]:
df.shape

(1000, 3)

In [20]:
df.to_csv('scrapped_books.csv' , index=False)